In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd


df = pd.read_csv("../../dataset/set_01_02_03_04_0_0_0_new.csv", encoding='utf-8').dropna()
df.columns = ['label', 'text']
df.head(5)

,label,text
0,1,number urgent this is the number nd attempt to...
1,1,free for number st week no number nokia tone n...
2,1,urgent call number from landline your compleme...
3,1,winner as a valued network customer you have b...
4,1,okmail dear dave this is your final notice to ...


In [2]:
df['label'].value_counts()

label
1    17598
0    15444
Name: count, dtype: int64

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['label'], stratify=df['label'])

In [4]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [5]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [6]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [7]:
model = tf.keras.Sequential([tf.keras.Model(inputs=[text_input], outputs = [l])])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 1)                 109483010 
                                                                 
Total params: 109,483,010
Trainable params: 769
Non-trainable params: 109,482,241
_________________________________________________________________


In [9]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [10]:
model.fit(X_train, y_train, epochs=1)

425/425 [==============================] - 72s 153ms/step - loss: 0.2078 - accuracy: 0.9249 - precision: 0.9243 - recall: 0.9803


In [11]:
#Evaluate data
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

import numpy as np
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

142/142 [==============================] - 22s 152ms/step


array([1, 1, 1, ..., 1, 1, 1])

In [12]:
sample_dataset = [
 'You can win a lot of money, register in the link below',
 'You have an iPhone 10, spin the image below to claim your prize and it will be delivered in your door step',
 'You have an offer, the company will give you 50% off on every item purchased.',
 'Hey Bravin, dont be late for the meeting tomorrow will start lot exactly 10:30 am',
 'See you monday, we have alot to talk about the future of this company .'
]
model.predict(sample_dataset)

1/1 [==============================] - 1s 638ms/step


array([[0.17571129],
       [0.37621504],
       [0.71006036],
       [0.60523415],
       [0.16136725]], dtype=float32)

In [18]:
from keras.models import load_model

model.save('model.h5')

In [24]:
#Test load model from save
loaded_model = load_model('model.h5',custom_objects={'KerasLayer': hub.KerasLayer})


In [31]:
#Test predict from loaded model
test_dataset=[
    'test'
]
array_result = loaded_model.predict(test_dataset)
print(array_result)

1/1 [==============================] - 0s 46ms/step
[[0.06537469]]


In [38]:
import json
result = str(array_result.tolist()).lstrip('[').rstrip(']')
print()
json_str = json.dumps()
#json_str.update(json_str.pop("Parent"))
#print(json_str)

0.06537468731403351
